In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras import applications
from keras.layers import Dense,GlobalAveragePooling2D
from keras.models import Model

import numpy as np
import matplotlib.pyplot as plt
import os 
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DATADIR = "train_images"
CATEGORIES = [str(i) for i in range(121)]

In [ ]:
IMG_SIZE = 95

plt.imshow(new_array, cmap = "gray")

In [ ]:
IMG_SIZE = 95

training_data = []

def create_training_data():
    for index, category in enumerate(CATEGORIES,0):
        path = os.path.join(DATADIR, category) 
        class_num = index
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img))
            new_array = cv2.resize(img_array,(IMG_SIZE, IMG_SIZE))
            training_data.append([new_array, class_num])
            
create_training_data()
        

In [ ]:
import random 

random.shuffle(training_data)

In [ ]:

X = []
y = []

class_weight = {}

for features, label in training_data:
    
    X.append(features)
    
    one_hot = [0 for i in range(121)]
    one_hot[label] = 1
    
    y.append(one_hot)
    
    if label in class_weight:
      class_weight[label] += 1.0
      
    else:
      class_weight[label] = 1.0
      
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3) 
y = np.array(y)

# create class weights
for class_num in class_weight:
  class_weight[class_num] = 1 - (class_weight[class_num] / len(y))
  

In [ ]:
X.shape

In [ ]:
X = X/255.0 # normalize

In [ ]:
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (95,95,3))

In [ ]:
x = model.output
x = Flatten()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(121, activation="softmax")(x)

In [ ]:
model=Model(inputs=base_model.input,outputs=predictions)

In [ ]:
for layer in model.layers:
    print(layer.name)

In [ ]:
model.compile(loss = "categorical_crossentropy",
             optimizer = "adam",
             metrics = ["accuracy"])

In [ ]:
model = Sequential()
N_Classes = 121

# model.add(InputLayer())
model.add(Conv2D(64,(3,3),padding = "same", input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding = "same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding = "same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(100))
model.add(Dense(N_Classes, activation='softmax'))

model.compile(loss = "categorical_crossentropy",
             optimizer = "adam",
             metrics = ["accuracy"])

In [ ]:
model.fit(X, y, batch_size = 64, epochs = 500, validation_split = 0.1)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator 

train_generator = ImageDataGenerator(
    
    rotation_range = 360,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest'

)

val_generator = ImageDataGenerator(
    
    horizontal_flip = True,
    vertical_flip = True,

)

train_generator.fit(X) # computes quantities required for featurewise normalization

N_epochs = 3
Batch_size = 64

# fits the model on batches with real-time data augmentation:
model.fit_generator(train_generator.flow(X, y, batch_size = Batch_size),
                    steps_per_epoch=len(X) / Batch_size, epochs = N_epochs)
    

In [ ]:
model.fit_generator(train_generator.flow(X, y, batch_size = Batch_size),
                    steps_per_epoch=len(X) / Batch_size, epochs = N_epochs)
    

# Test and Write

In [ ]:
from PIL import Image

directory_in_str = "drive/My Drive/AML_plankton_project/test_images/test"


results = list()


for file in os.listdir(directory_in_str):
    if file.endswith(".jpg"):
        
        filename = os.fsdecode(file)
        image_path = os.path.join(directory_in_str, filename)
        
        img_array = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        new_array = cv2.resize(img_array,(IMG_SIZE, IMG_SIZE))
        norm_array = new_array / 255
        norm_array = np.array(norm_array).reshape(-1,IMG_SIZE, IMG_SIZE, 1) 
        prediction = model.predict_classes(norm_array)
        
        results.append([filename, prediction[0]])
        
                  
test_results = pd.DataFrame(results, columns= ["image","class"])

In [ ]:
test_results.to_csv('output_eerste_keras.csv', index=False)

In [ ]:
from google.colab import files

files.download("output_eerste_keras.csv")